In [2]:
#26/8/24 Creates a K fold KNN model for each file in a folder provided they are padel
import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '999'
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import nnmetrics
from Function_Repo import setbalance
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Padel/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    threshlist = []
    finalresults = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        
        keras.utils.set_random_seed(seed)
        tf.config.experimental.enable_op_determinism()
        length = len(xtrain.columns)
        sampcount = len(ytrain)
        temp = length
        # Encoder Creation
        enc_in = keras.Input(shape=(length, 1))
        x = keras.layers.Flatten()(enc_in)
        # Additional encoder layers
        
        layer1 = math.sqrt(3*sampcount) + 2*(math.sqrt(sampcount/(3)))
        x = keras.layers.Dense(layer1, activation="relu")(x)
        
        layer2 = math.sqrt(sampcount / 3)
        x = keras.layers.Dense(layer2, activation="relu")(x)
        
        enc_out = keras.layers.Dense(1, activation="sigmoid")(x)
        autoencoder = keras.Model(enc_in, enc_out, name='autoencoder')
        # Optimiser
        
        optimise = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

        autoencoder.compile(optimise, loss='binary_crossentropy')
        #autoencoder.summary()
        
        ytrain = np.array(ytrain)
        epochs = 500
        batchsize = 16
        losshistory = []

        minloss = 1
        bestepoch = 0
        for epoch in range(epochs):
            verb = 0
            logs = autoencoder.fit(
            xtrain,
            ytrain,
            verbose = verb,
            epochs=1,
            batch_size=batchsize),
            validation_split=0.2,

            #autoencoder.save(str(f"models/AE-{epoch+1}.keras"))
            loss = logs[0].history['loss'][0]
            losshistory.append(loss)
            if loss < minloss:
                minloss = loss
                bestepoch = epoch

        model_list.append(autoencoder)
    
        preds = autoencoder.predict(xtest)

        epochcount = range(epochs)

        acclist = []
        thresholdlist = []
        maxval = 0
        bestthresh = 0
        for threshold in np.arange(0, 1, 0.01):
            newval = nnmetrics(preds, ytest, threshold)[12]
            acclist.append(newval)
            thresholdlist.append(threshold)
            if newval > maxval:
                maxval = newval
                bestthresh = threshold
        threshlist.append(bestthresh)
        testdat.append(maxval)
        y_values = acclist
        x_values = thresholdlist
        preds = autoencoder.predict(xvalid)
        results = nnmetrics(preds, yvalid, bestthresh)
        validlist.append(results)
        print('for fold', split + 1, 'optimal test mcc of', maxval, 'validation mcc of', results[12])

        iteration = iteration + 1

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        

    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []

    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        fp = np.asarray(fp).astype('float32')
        #appends each model's prediction to a list
        predlist = []
        for model in model_list:
            preds = model.predict(fp, verbose = 0)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        loops = 0
        for modelpreds in predlist:
            if modelpreds < threshlist[loops]:
                zercount = zercount + 1
        loops = loops + 1

        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)

    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    del autoencoder
    gc.collect
    
    sets += 1

For dataset Hepatotoxicity_PaDEL.csv
6/6 [==============================] - 0s 672us/step
for fold 1 optimal test mcc of 0.3138213482007467 validation mcc of 0.16627124251422518
6/6 [==============================] - 0s 1ms/step
for fold 2 optimal test mcc of 0.20534304002280163 validation mcc of 0.19764318885553456
6/6 [==============================] - 0s 527us/step
for fold 3 optimal test mcc of 0.2682167093113806 validation mcc of 0.2121498079533411
6/6 [==============================] - 0s 643us/step
for fold 4 optimal test mcc of 0.28868869091849997 validation mcc of 0.24689699641850607
6/6 [==============================] - 0s 565us/step
for fold 5 optimal test mcc of 0.2598028297347748 validation mcc of 0.11970383374705876

validation metrics of:
positives in data 249
negatives in data 226
net accuracy = 0.6757894736842105
mcc = 0.3612083148139321
For dataset NR-PPAR-gamma_PaDEL.csv
1/1 [==============================] - 0s 10ms/step
for fold 1 optimal test mcc of 0.30031248756

1/1 [==============================] - 0s 9ms/step
for fold 1 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 2 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 3 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 4 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 5 optimal test mcc of 0 validation mcc of 0

validation metrics of:
positives in data 0
negatives in data 0
net accuracy = 0
mcc = 0
For dataset NR-ER-LBD_PaDEL.csv
2/2 [==============================] - 0s 747us/step
for fold 1 optimal test mcc of 0.3055970846781293 validation mcc of 0.2619924972972864
2/2 [==============================] - 0s 806us/step
for fold 2 optimal test mcc of 0.44565217391304346 validation mcc of 0.11359236684941296
2/2 [==============================] - 0s 811us/step
for fold 3 optimal test 

3/3 [==============================] - 0s 605us/step
for fold 4 optimal test mcc of 0.10519514998302663 validation mcc of 0.1778277701407802
3/3 [==============================] - 0s 578us/step
for fold 5 optimal test mcc of 0.4248915789631691 validation mcc of 0.1813041603738673

validation metrics of:
positives in data 94
negatives in data 94
net accuracy = 0.5425531914893617
mcc = 0.0880237696271323
For dataset NR-ER_PaDEL.csv
5/5 [==============================] - 0s 573us/step
for fold 1 optimal test mcc of 0.27754336175689315 validation mcc of 0.0893401450148153
5/5 [==============================] - 0s 563us/step
for fold 2 optimal test mcc of 0.16157887025338957 validation mcc of 0.20927539886411303
5/5 [==============================] - 0s 512us/step
for fold 3 optimal test mcc of 0.20459353067783365 validation mcc of 0.21671636552601772
5/5 [==============================] - 0s 504us/step
for fold 4 optimal test mcc of 0.2625212638029782 validation mcc of 0.3114447481729269
5

5/5 [==============================] - 0s 593us/step
for fold 4 optimal test mcc of 0.4368839052455692 validation mcc of 0.4511777625672017
5/5 [==============================] - 0s 527us/step
for fold 5 optimal test mcc of 0.3837427456522708 validation mcc of 0.32843200034363534

validation metrics of:
positives in data 161
negatives in data 217
net accuracy = 0.7116402116402116
mcc = 0.45406068756763107


In [3]:
finalresults

[0.45406068756763107]

In [ ]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Data/Model_Outputs/Fingerprint/NeuralNet_Fingerprint_Results')
resultframe

In [5]:
namelist

['NR-ER_Chemopy.csv',
 'Developmental_Toxicity_Chemopy.csv',
 'NR-aromatase_Chemopy.csv',
 'SR-ATAD5_Chemopy.csv',
 'SR-ARE_Chemopy.csv',
 'SR-p53_Chemopy.csv',
 'Cardiotoxicity-1_Chemopy.csv',
 'SR-HSE_Chemopy.csv',
 'Respiratory_Toxicity_Chemopy.csv',
 'Carcinogenicity_Chemopy.csv',
 'Cardiotoxicity-30_Chemopy.csv',
 'Cardiotoxicity-10_Chemopy.csv',
 'Cardiotoxicity-5_Chemopy.csv',
 'NR-AR-LBD_Chemopy.csv',
 'Ames_Mutagenicity_Chemopy.csv',
 'Hepatotoxicity_Chemopy.csv',
 'NR-AR_Chemopy.csv',
 'Reproductive_Toxicity_Chemopy.csv',
 'NR-ER-LBD_Chemopy.csv',
 'NR-PPAR-gamma_Chemopy.csv',
 'NR-AhR_Chemopy.csv',
 'SR-MMP_Chemopy.csv']